In [4]:
import requests
html = requests.get('http://google.com/')


In [10]:
from bs4 import BeautifulSoup

soup =  BeautifulSoup(html.content, 'html.parser')

In [13]:
soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en-IN"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/><title>Google</title><script nonce="lBCDWKuqnf-nD7RNzRHeSA">(function(){var _g={kEI:'0qdVZp7DIMKv5OUP8P22wAM',kEXPI:'0,3700284,1026,77,19,448506,58573,31560,2872,2891,11754,77401,230,17935,89292,6651,49751,2,16737,23024,6699,41949,54821,2913,2,2,1,24626,2006,8155,23351,22435,9779,42459,20198,33566,39613,3030,15816,1804,7734,13278,27705,40382,10674,5303,5209427,2357,916,166,431,38,5991771,2839782,386,1,38,27981928,1007,13290,2375,43887,3,318,4,1281,3,2121778,2585,23029351,8163,10336,2709,8027,8639,28025,36870,1922,8589,2370,6407,2863,5185,5798,15164,4792,2,3187,201,390,5538,14786,624,1514,4,10398,16175,10085,7766,3822,4797,1954,155,2,397,2085,13503,7737,7,5156,3,1431,2,1426,1113,968,539,3093,206,122,405,2263,549,4,123,2881,272,433,3561,2

In [7]:
import requests
from bs4 import BeautifulSoup

# Function to scrape data from a specific section page
def scrape_section_page(section_url, class_names):
    response = requests.get(section_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    section_posts = []
    for class_name in class_names:
        news_list_section = soup.find_all('div', class_=class_name)
        for news_div in news_list_section:
            news_links = news_div.find_all('a', href=True, title=True)
            for link in news_links:
                post_title = link['title'].strip()
                post_url = section_url + link['href']
                section_posts.append({'title': post_title, 'url': post_url})
    
    return section_posts

# Function to scrape content from a post page
def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove unwanted elements
    for auth_div in soup.find_all('div', class_='auth-name-dt'):
        auth_div.decompose()
    for author_div in soup.find_all('div', class_='h-author'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='col-md-4'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='d-mags'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='d-social'):
        author_div.decompose()

    # Extract content
    paragraphs = soup.find_all('p')
    content = "\n\n".join([p.text.strip() for p in paragraphs])
    
    return content

# Main function to scrape the data
def main(language_urls, main_class, khabar_classes, weather_class, khati_class):
    all_results = {}

    for lang, urls in language_urls.items():
        print(f"Scraping data for language: {lang}")
        
        base_url = urls["base"]
        khati_url = urls["khati_badi"]
        
        # Scrape the main page
        main_posts = scrape_section_page(base_url, main_class)
        khabar_posts = scrape_section_page(base_url, khabar_classes)

        # Scrape the weather page if available
        if "weather" in urls:
            weather_posts = scrape_section_page(urls["weather"], weather_class)
        else:
            weather_posts = []

        # Scrape the Khati Badi page
        khati_posts = scrape_section_page(khati_url, khati_class)

        # Set to track processed URLs
        processed_urls = set()

        # Dictionary to store the results for this language
        results = {
            "Main Page Posts": [],
            "Khabar Section Posts": [],
            "Weather Posts": [],
            "Khati Badi Posts": []
        }

        # Process main page posts
        for post in main_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Main Page Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])     

        # Process "Khabar" section posts
        for post in khabar_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khabar Section Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process weather posts
        for post in weather_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Weather Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process Khati Badi posts
        for post in khati_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khati Badi Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        all_results[lang] = results

    return all_results

# Define the base URLs and other URLs for each language
language_urls = {
    "hindi": {
        "base": "https://hindi.krishijagran.com/",
        "weather": "https://hindi.krishijagran.com/weather",
        "khati_badi": "https://hindi.krishijagran.com/farm-activities"
    },
    "english": {
        "base": "https://krishijagran.com/",
        "khati_badi": "https://krishijagran.com/agriculture-world"
    },
    # Add other languages and URLs here
}

# Define the class names for scraping
main_class = 'home-top-l'
khabar_classes = ['home-top-news-lst', 'other-khabar-class']
weather_class = 'news-list-wide shadow-sm'
khati_class = 'news-list-wide shadow-sm'

# Start the scraping process and get the results
scraped_data = main(language_urls, main_class, khabar_classes, weather_class, khati_class)
print(scraped_data)


Scraping data for language: hindi
Scraping data for language: english
{'hindi': {'Main Page Posts': [], 'Khabar Section Posts': [{'title': 'IARI ने किसानों को दिया बड़ा तोहफा, कम पानी, कम लागत में उगायें बासमती की नई खरपतवार नाशी किस्म!', 'link': 'https://hindi.krishijagran.com//news/iari-pusa-released-two-new-varieties-of-basmati-paddy-pusa-basmati1979-and-pusa-basmati1985-latest-news/', 'content': "भारतीय कृषि अनुसंधान संस्थान (IARI) पूसा ने बीते कल यानी की 22 मई के दिन बासमती धान की दो नई किस्में जारी की. इसे पूसा संस्थान के डॉ बीपी पाल ओडोटोरियम में लॉन्च किया गया है. यहां जानें पूरी जानकारी...\n\nभारतीय कृषि अनुसंधान संस्थान (IARI) पूसा ने बुधवार को बासमती धान की दो नई किस्में बाजार में उतारी हैं, जिसका लांच पूसा संस्थान के डॉ बीपी पाल ओडोटोरियम से किया गया. इस का विमोचन IARI निदेशक डॉ ऐके सिंह व् कृषि आयुक्त डॉ पी के सिंह द्वारा किया गया. आपको बता दें धान की ये किस्म किसानों की आय को बढ़ाने के साथ-साथ पानी की 35% तक बचत करेंगी. इसके आलावा इस किस्म में खरपतवार नाशी गुण भी शामिल किय

In [ ]:
["tamil": {
        "base": "https://tamil.krishijagran.com/",
        "main_class": ["home-top-l"],
        "news_classes": ["home-top-news-lst"],
        "weather_class": ["no weather"],
        "khati_badi_class": ["home-2-3-lst"],
    },
"marathi": {
        "base": "https://marathi.krishijagran.com/",
        "main_class": ["home-top-l"],
        "news_classes": ["home-top-news-lst"],
        "weather_class": ["weather-home mt-5 mb-3"],
        "khati_badi_class": ["home-2-3-lst"],
    },]


In [6]:
import requests
from bs4 import BeautifulSoup
 
# Function to scrape data from a specific section page
def scrape_section_page(section_url, class_name):
    response = requests.get(section_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    section_posts = []
    news_list_section = soup.find_all('div', class_=class_name)
    for news_div in news_list_section:
        news_links = news_div.find_all('a', href=True, title=True)
        for link in news_links:
            post_title = link['title'].strip()
            post_url = section_url + link['href']
            section_posts.append({'title': post_title, 'url': post_url})
    
    return section_posts

# Function to scrape content from a post page
def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove unwanted elements
    for unwanted_div in soup.find_all(['div', 'span', 'label']):
        unwanted_div.decompose()

    # Extract content
    paragraphs = soup.find_all('p')
    content = "\n\n".join([p.text.strip() for p in paragraphs])
    
    return content

# Main function to scrape the data
def main(base_url, weather_url, khati_url, main_class, khabar_class, weather_class, khati_class):
    # Scrape the main page
    main_posts = scrape_section_page(base_url, main_class)
    khabar_posts = scrape_section_page(base_url, khabar_class)

    # Scrape the weather page
    weather_posts = scrape_section_page(weather_url, weather_class)

    # Scrape the Khati Badi page
    khati_posts = scrape_section_page(khati_url, khati_class)

    # Set to track processed URLs
    processed_urls = set()

    # Dictionary to store the results
    results = {
        "Main Page Posts": [],
        "Khabar Section Posts": [],
        "Weather Posts": [],
        "Khati Badi Posts": []
    }

    # Process main page posts
    for post in main_posts:
        if post['url'] not in processed_urls:
            content = scrape_post_content(post['url'])
            results["Main Page Posts"].append({
                "title": post['title'],
                "link": post['url'],
                "content": content
            })
            processed_urls.add(post['url'])

    # Process "Khabar" section posts
    for post in khabar_posts:
        if post['url'] not in processed_urls:
            content = scrape_post_content(post['url'])
            results["Khabar Section Posts"].append({
                "title": post['title'],
                "link": post['url'],
                "content": content
            })
            processed_urls.add(post['url'])

    # Process weather posts
    for post in weather_posts:
        if post['url'] not in processed_urls:
            content = scrape_post_content(post['url'])
            results["Weather Posts"].append({
                "title": post['title'],
                "link": post['url'],
                "content": content
            })
            processed_urls.add(post['url'])

    # Process Khati Badi posts
    for post in khati_posts:
        if post['url'] not in processed_urls:
            content = scrape_post_content(post['url'])
            results["Khati Badi Posts"].append({
                "title": post['title'],
                "link": post['url'],
                "content": content
            })
            processed_urls.add(post['url'])

    return results

# Define the URLs and class names for scraping
base_url = "https://hindi.krishijagran.com/"
weather_url = "https://hindi.krishijagran.com/weather"
khati_url = "https://hindi.krishijagran.com/farm-activities"
main_class = 'home-top-l'
khabar_class = 'home-top-news-lst'
weather_class = 'news-list-wide shadow-sm'
khati_class = 'news-list-wide shadow-sm'
    
# Start the scraping process and get the results
scraped_data = main(base_url, weather_url, khati_url, main_class, khabar_class, weather_class, khati_class)
print(scraped_data)


KeyboardInterrupt: 

In [ ]:
    import requests
from bs4 import BeautifulSoup

# Function to scrape data from a specific section page
def scrape_section_page(section_url, class_names):
    response = requests.get(section_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    section_posts = []
    for class_name in class_names:
        news_list_section = soup.find_all('div', class_=class_name)
        for news_div in news_list_section:
            news_links = news_div.find_all('a', href=True, title=True)
            for link in news_links:
                post_title = link['title'].strip()
                post_url = section_url + link['href']
                section_posts.append({'title': post_title, 'url': post_url})
    
    return section_posts

# Function to scrape content from a post page
def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove unwanted elements
    for unwanted_div in soup.find_all(['div', 'span', 'label']):
        unwanted_div.decompose()

    # Extract content
    paragraphs = soup.find_all('p')
    content = "\n\n".join([p.text.strip() for p in paragraphs])
    
    return content

# Main function to scrape the data
def main(language_urls, main_class, khabar_classes, weather_class, khati_class):
    all_results = {}

    for lang, urls in language_urls.items():
        print(f"Scraping data for language: {lang}")
        
        base_url = urls["base"]
        weather_url = urls["weather"]
        khati_url = urls["khati_badi"]
        
        # Scrape the main page
        main_posts = scrape_section_page(base_url, main_class)
        khabar_posts = scrape_section_page(base_url, khabar_classes)

        # Scrape the weather page
        weather_posts = scrape_section_page(weather_url, weather_class)

        # Scrape the Khati Badi page
        khati_posts = scrape_section_page(khati_url, khati_class)

        # Set to track processed URLs
        processed_urls = set()

        # Dictionary to store the results for this language
        results = {
            "Main Page Posts": [],
            "Khabar Section Posts": [],
            "Weather Posts": [],
            "Khati Badi Posts": []
        }

        # Process main page posts
        for post in main_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Main Page Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process "Khabar" section posts
        for post in khabar_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khabar Section Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process weather posts
        for post in weather_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Weather Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process Khati Badi posts
        for post in khati_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khati Badi Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        all_results[lang] = results

    return all_results

# Define the base URLs and other URLs for each language
language_urls = {
    "hindi": {
        "base": "https://hindi.krishijagran.com/",
        "weather": "https://hindi.krishijagran.com/weather",
        "khati_badi": "https://hindi.krishijagran.com/kheti-badi"
    },
    "english": {
        "base": "https://krishijagran.com/",
        # "weather": "https://english.krishijagran.com/weather",
        "khati_badi": "https://krishijagran.com/agriculture-world"
    },
    # Add other languages and URLs here
}

# Define the class names for scraping
main_class = 'home-top-l'
khabar_classes = ['home-top-news-lst', 'home-2-3-lst']
weather_class = 'news-list-wide shadow-sm'
khati_class = 'news-list-wide shadow-sm'

# Start the scraping process and get the results
scraped_data = main(language_urls, main_class, khabar_classes, weather_class, khati_class)
print(scraped_data)


In [5]:
# hindi language
# ------hindi languagde----
# import requests
# from bs4 import BeautifulSoup

# # Define the base URL
# base_url = "https://hindi.krishijagran.com/"

# # Function to scrape data from the main page and "Khabar" section
# def scrape_main_page():
#     response = requests.get(base_url)
#     soup = BeautifulSoup(response.content, 'html.parser')
    
#     # Extract post names and links till "Fasal Bima Yojana"
#     main_posts = []
#     posts_section = soup.find('div', class_='home-top-wrap')
#     post_links = posts_section.find_all('a', href=True, title=True)
    
#     for link in post_links:
#         post_title = link['title'].strip()
#         post_url = base_url + link['href']
#         main_posts.append({'title': post_title, 'url': post_url})
#         # Stop scraping when "Fasal Bima Yojana" is found
#         if "Fasal Bima Yojana" in post_title:
#             break
    
#     # Scroll down to "Khabar" part
#     khabar_section = soup.find('div', class_='home-top-news-lst')
#     khabar_links = khabar_section.find_all('a', href=True, title=True)
    
#     khabar_posts = []
#     for link in khabar_links:
#         post_title = link['title'].strip()
#         post_url = base_url + link['href']
#         khabar_posts.append({'title': post_title, 'url': post_url})
    
#     # Combine both sets of posts into a single list
#     all_posts = main_posts + khabar_posts
    
#     return all_posts

# #Function to scrape content from a post page
# def scrape_post_content(post_url):
#     response = requests.get(post_url)
#     soup = BeautifulSoup(response.content, 'html.parser')
    
#     # Extract all paragraphs
#     paragraphs = soup.find_all('p')
#     content = "\n\n".join([p.text.strip() for p in paragraphs])
    
#     return content

#Main function to scrape the data
# def main():
#     # Scrape the main page
#     all_posts = scrape_main_page()
    
#     # Print all posts
#     print("All Posts:")
#     for post in all_posts:
#         content = scrape_post_content(post['url'])
#         print(f"Title: {post['title']}")
#         print(f"Content: {content} \n\n")
#         print("\n\n")

#Start the scraping process
# main()
#marathi langauge
#------marathi languagde----
import requests
from bs4 import BeautifulSoup

#Define the base URL
base_url = "https://marathi.krishijagran.com/"

#Function to scrape data from the main page and "Khabar" section
def scrape_main_page():
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract post names and links till "Fasal Bima Yojana"
    # main_posts = []
    posts = set()
    posts_section = soup.find('div', class_='home-top-wrap')
    post_links = posts_section.find_all('a', href=True, title=True)
    
    for link in post_links:
        post_title = link['title'].strip()
        post_url = base_url + link['href']
        # main_posts.append({'title': post_title, 'url': post_url})
        posts.add({'title': post_title, 'url': post_url})
        # Stop scraping when "Fasal Bima Yojana" is found
        if "Fasal Bima Yojana" in post_title:
            break
    # Scroll down to "Khabar" part
    khabar_section = soup.find('div', class_='home-top-news-lst')
    khabar_links = khabar_section.find_all('a', href=True, title=True)
    
    khabar_posts = []
    for link in khabar_links:
        post_title = link['title'].strip()
        post_url = base_url + link['href']
        khabar_posts.append({'title': post_title, 'url': post_url})
    
    # Combine both sets of posts into a single list
    # all_posts = main_posts

    return all_posts

#Function to scrape content from a post page
def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract all paragraphs
    paragraphs = soup.find_all('p')
    content = "\n\n".join([p.text.strip() for p in paragraphs])
    
    return content

#Main function to scrape the data
def main():
    # Scrape the main page
    all_posts = scrape_main_page()
    posts = set()
    # Print all posts
    print("All Posts:")
    for post in all_posts:
        content = scrape_post_content(post['url'])
        #add the content to posts avoid duplication
        posts.add(content)
        # print(f"Title: {post['title']}")
        # print(f"Content: {content} \n\n")
        # print("\n\n")
        # print(f"Title: {post['title']}")
        # print(f"Content: {content} \n\n")
        # print("\n\n")
    return posts

# Start the scraping process
data = main()


TypeError: unhashable type: 'dict'

In [ ]:
data

{'',
 'आमच्या प्रिंट आणि डिजिटल मासिकांची सदस्यता घ्या\n\nसोशल मीडियावर आमच्याशी संपर्क साधा:\n\n\n\nकृषी आणि शेतकरी कल्याण मंत्रालयाने कृषी पायाभूत सुविधा निधी योजना सुरू केली आहे. या योजनेचा मुख्य उद्देश म्हणजे कोल्ड स्टोअर्स, वेअरहाऊसिंग, सायलो, पॅकिंग युनिट्स, असेईंग/ग्रेडिंग,…\n\nया योजनेत धरणातील साठलेला गाळ काढून टाकून त्यांची साठवण क्षमता वाढविण्याबरोबरच या गाळाचा वापर शेतशिवारामध्ये करून, शेतकऱ्यांना त्यांच्या शेतजमिनीची गुणवत्ता वाढवण्यासाठी फायदा होईल ज्यामुळे उत्पादन…\n\nमहिला घरी राहूनच स्कूल बॅग बनवू शकतात आणि तयार करू शकतात. आजकाल विविध प्रकारच्या स्कूल बॅग बाजारात उपलब्ध आहेत. हा व्यवसाय वर्षभर चालतो आणि शाळेच्या…\n\nवयोमानानुसार निवृत्ती स्वीकारावी लागते. त्यामुळं निवृत्तीवेतन हाच अनेकांसाठी जगण्याचा आधार ठरतो. म्हणूनच सुरवातीपासूनच आम्ही या विषयावर सुरवातीपासून संवेदनशील राहीलो आहोत. निवृत्तीनंतर शासकीय कर्मचाऱ्यांना सुरक्षित व…\n\nघरकुलापासून कोण वंचित आहे, ‘ड’ यादीत नाव नाही आणि जो पात्र आहे, अशा सर्वांना घरकुल देण्यात येईल. चंद्रपूर जिल्ह्यातील एकही आदिवासी बांधव घ

In [15]:
lis = list(data)
for i in lis[1:]:
    print(i)
    break

आमच्या प्रिंट आणि डिजिटल मासिकांची सदस्यता घ्या

सोशल मीडियावर आमच्याशी संपर्क साधा:



कृषी आणि शेतकरी कल्याण मंत्रालयाने कृषी पायाभूत सुविधा निधी योजना सुरू केली आहे. या योजनेचा मुख्य उद्देश म्हणजे कोल्ड स्टोअर्स, वेअरहाऊसिंग, सायलो, पॅकिंग युनिट्स, असेईंग/ग्रेडिंग,…

या योजनेत धरणातील साठलेला गाळ काढून टाकून त्यांची साठवण क्षमता वाढविण्याबरोबरच या गाळाचा वापर शेतशिवारामध्ये करून, शेतकऱ्यांना त्यांच्या शेतजमिनीची गुणवत्ता वाढवण्यासाठी फायदा होईल ज्यामुळे उत्पादन…

महिला घरी राहूनच स्कूल बॅग बनवू शकतात आणि तयार करू शकतात. आजकाल विविध प्रकारच्या स्कूल बॅग बाजारात उपलब्ध आहेत. हा व्यवसाय वर्षभर चालतो आणि शाळेच्या…

वयोमानानुसार निवृत्ती स्वीकारावी लागते. त्यामुळं निवृत्तीवेतन हाच अनेकांसाठी जगण्याचा आधार ठरतो. म्हणूनच सुरवातीपासूनच आम्ही या विषयावर सुरवातीपासून संवेदनशील राहीलो आहोत. निवृत्तीनंतर शासकीय कर्मचाऱ्यांना सुरक्षित व…

घरकुलापासून कोण वंचित आहे, ‘ड’ यादीत नाव नाही आणि जो पात्र आहे, अशा सर्वांना घरकुल देण्यात येईल. चंद्रपूर जिल्ह्यातील एकही आदिवासी बांधव घरापासून वंचित राहणार…

In [3]:
from bs4 import BeautifulSoup

In [17]:
from bs4 import BeautifulSoup

html = """
<div class="news-list-wide shadow-sm">
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/" title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">
                                        <h2 title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत</h2>
                                        <p>किसानों को कम लागत और कम समय में&nbsp;धान की अच्छी पैदावार देने के उद्देश्य से&nbsp;पूसा संस्थान ने&nbsp;धान की 2 नई किस्में जारी की हैं. संस्थान ने&nbsp;कल…</p>
                                    </a>
                                </div>
                                ...
                            </div>
                            ...
                </div>
"""

soup = BeautifulSoup(html, 'html.parser')
links = soup.find_all('a')

for link in links:
    title = link.get('title')
    href = link.get('href')
    if title and href:
        print(f"{title} - {href}")

पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत - /farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/


In [ ]:
site = "https://hindi.krishijagran.com/"


In [7]:
from bs4 import BeautifulSoup

html_content = """<div class="news-list-wide shadow-sm">
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/" title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">
                                        <h2 title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत</h2>
                                        <p>किसानों को कम लागत और कम समय में&nbsp;धान की अच्छी पैदावार देने के उद्देश्य से&nbsp;पूसा संस्थान ने&nbsp;धान की 2 नई किस्में जारी की हैं. संस्थान ने&nbsp;कल…</p>
                                    </a>
                                </div>
                                ... (remaining HTML content)
                            </div>
                </div>"""

soup = BeautifulSoup(html, 'html.parser')
urls = [a['href'] for a in soup.select('div.news-info a')]

print(urls)

........................................................................



['/farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/', '/farm-activities/ragi-madua-with-scientific-techniques-farmers-earn-good-profits-by-cultivating-ragi-ki-kheti/', '/farm-activities/chilli-cultivate-these-5-improved-varieties-of-green-chilli-in-june/', '/farm-activities/context-of-climate-change-farmers-can-do-direct-sowing-of-paddy-with-advanced-technology/', '/farm-activities/how-to-start-dragon-fruit-farming-in-india-dragon-fruit-different-varieties-cost-yield-and-profit/', '/farm-activities/maize-farming-in-kharif-season-is-more-profitable-than-paddy-farming/', '/farm-activities/peanut-farming-tips-farmers-will-earn-more-profit-mungfali-ki-kheti/', '/farm-activities/peak-borer-infection-in-sugarcane-crop-damage-and-prevention/', '/farm-activities/black-maize-cultivation-farmers-earn-huge-income-one-corn-cost-200-rupees/', '/farm-activities/top-4-varieties-of-soybean-in-india-soyabean-ki-kheti/', '/farm-activities/how-to-buy-nsc-basmati-rice-s

In [8]:
from bs4 import BeautifulSoup

html_content = """<div class="h">

<ol class="list-unstyled">

<li>

<a href="/" title="Home"><small>Home</small></a>

</li>

<li>

<h2 class="n-cat" title="">

<a href="/farm-activities/" title="">खेती-बाड़ी</a>

</h2>

</li>

</ol>

<h1 title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी, बंपर मिलेगी उपज, जानें अन्य खासियत">पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी, बंपर मिलेगी उपज, जानें अन्य खासियत</h1>

<p>किसानों को कम लागत और कम समय में&nbsp;धान की अच्छी पैदावार देने के उद्देश्य से&nbsp;पूसा संस्थान ने&nbsp;धान की 2 नई किस्में जारी की हैं. संस्थान ने&nbsp;कल यानी 22 मई, 2024 को रॉबिनोवीड बासमती चावल की किस्मों, पूसा बासमती 1979 और पूसा बासमती 1985 की किस्मों को जारी किया है.</p>

<div class="n-meta d-flex flex-column">

<div class="auth-dt">

<div class="n-auth">

<div class="auth-name-dt">

<span>

<a href="https://hindi.krishijagran.com/authors/mohit-naagar/" title="मोहित नागर">

मोहित नागर

</a>

</span>

<label class="pub-date">Updated on 23 May, 2024 10:43 AM IST <i class="bi bi-info-circle-fill" data-toggle="tooltip" data-placement="top" title="Published on 23 May, 2024 10:06 AM IST"></i></label>

</div>

</div>

</div>

<div class="share-chat-vuukle" style="height:52px">

<div class="vuukle-powerbar"><iframe height="0" name="h-sharebar-0" id="h-sharebar-0" title="Vuukle Sharebar Widget" frameborder="0" scrolling="no" allow="clipboard-write" style="width: 100%; min-width: 100%; min-height: 45px; height: 49px;" class="gm-lazy gm-added" src="https://cdn.vuukle.com/widgets/sharebar.html?version=2.21.8"></iframe></div>

</div>

</div>

</div>"""

soup = BeautifulSoup(html_content, 'html.parser')

# Extract title
title = soup.find('h1').text.strip()

# Extract description
description = soup.find('p').text.strip()

print("Title:", title)
print("Description:", description)


Title: पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी, बंपर मिलेगी उपज, जानें अन्य खासियत
Description: किसानों को कम लागत और कम समय में धान की अच्छी पैदावार देने के उद्देश्य से पूसा संस्थान ने धान की 2 नई किस्में जारी की हैं. संस्थान ने कल यानी 22 मई, 2024 को रॉबिनोवीड बासमती चावल की किस्मों, पूसा बासमती 1979 और पूसा बासमती 1985 की किस्मों को जारी किया है.


In [12]:
from bs4 import BeautifulSoup

# html_doc = """
# <documents>
# <document index="1">
# <source>paste.txt</source>
# <document_content><article class="d-flex flex-column">
                            
                
    
#     <link rel="preload" as="image" fetchpriority="high" href="https://kjhindi.gumlet.io/media/87723/pusa-institute-new-delhi.jpg">
#     <figure>
#                 <!-- Check if the request is from FB-RSS-->
#                         <img alt="भारतीय कृषि अनुसंधान संस्थान,नई दिल्ली (Picture Credit - Krishi Jagran)" class="img-fluid ls-is-cached lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io/media/87723/pusa-institute-new-delhi.jpg?w=700&amp;dpr=1.3" width="1280" height="720" fetchpriority="high" loading="lazy">

#             <figcaption class="caption">भारतीय कृषि अनुसंधान संस्थान,नई दिल्ली (Picture Credit - Krishi Jagran)</figcaption>
#     </figure>


                
    
# <!-- CONTENT GOES HERE -->

                            
#                             <div class="d-flex flex-column">
#                             <span><strong>English Summary: </strong> pusa discovered two new varieties of Basmati paddy get bumper yield</span>
#                             <span><strong>Published on: </strong>23 May 2024, 10:40 AM IST</span>
#                         </div>
                        
#                         </article></document_content>
# </document>
# </documents>
# """

soup = BeautifulSoup(html_doc, 'html.parser')
article = soup.find('article')
content = article.get_text(separator='\n', strip=True)

print(content)

भारतीय कृषि अनुसंधान संस्थान,नई दिल्ली (Picture Credit - Krishi Jagran)
भारत के
किसानों को कम लागत और कम समय में
धान की अच्छी पैदावार देने के उद्देश्य से
भारतीय कृषि अनुसंधान संस्थान दिल्ली ने दो धान की नई किस्में जारी की हैं. बता दें, पूसा संस्थान ने
कल यानी 22 मई, 2024 को रॉबिनोवीड बासमती चावल की किस्मों, पूसा बासमती 1979 और पूसा बासमती 1985 की किस्मों को जारी किया है, जो सीधे बीज वाले चावल की खेती के लिए इमेजेथापायर 10 प्रतिशत एसएल के प्रति सहनशील हैं. इस संदर्भ में IARI, दिल्ली के निदेशक डॉ.अशोक कुमार सिंह ने बताया कि, उत्तर-पश्चिमी भारत में चावल की खेती में प्रमुख चिंताओं में (A) घटता जल स्तर (B) चावल की रोपाई के लिए मजदूरों की कमी और (C) रोपाई के दौरान बाढ़ की स्थिति में ग्रीनहाउस गैस, मीथेन का उत्सर्जन शामिल है.
पूसा संस्थान द्ववारा विकसीत सीधी बुआई वाला चावल इन सभी चिंताओं का समाधान है. डीएसआर के तहत खरपतवार एक बड़ी समस्या है जिसे डीएसआर को सफल बनाने के लिए संबोधित करने की आवश्यकता है. इस दिशा में, ICAR-IARI, नई दिल्ली में ठोस अनुसंधान से रोबिनोविड बासमती चावल की दो किस्मों, पू

In [11]:
html_doc = """<article class="d-flex flex-column">
                            
                
    
    <link rel="preload" as="image" fetchpriority="high" href="https://kjhindi.gumlet.io/media/87723/pusa-institute-new-delhi.jpg">
    <figure>
                <!-- Check if the request is from FB-RSS-->
                        <img alt="भारतीय कृषि अनुसंधान संस्थान,नई दिल्ली (Picture Credit - Krishi Jagran)" class="img-fluid ls-is-cached lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io/media/87723/pusa-institute-new-delhi.jpg?w=700&amp;dpr=1.3" width="1280" height="720" fetchpriority="high" loading="lazy">

            <figcaption class="caption">भारतीय कृषि अनुसंधान संस्थान,नई दिल्ली (Picture Credit - Krishi Jagran)</figcaption>
    </figure>


                
    
<p><span>भारत के </span><span>किसानों को कम लागत और कम समय में&nbsp;</span><span>धान की अच्छी पैदावार देने के उद्देश्य से&nbsp;</span><span></span><span>भारतीय कृषि अनुसंधान संस्थान दिल्ली ने&nbsp;दो धान की नई किस्में जारी की हैं. बता दें, पूसा संस्थान ने&nbsp;</span>कल यानी 22 मई, 2024 को रॉबिनोवीड बासमती चावल की किस्मों, पूसा बासमती 1979 और पूसा बासमती 1985 की किस्मों को जारी किया है, जो&nbsp;सीधे बीज वाले चावल की खेती के लिए इमेजेथापायर 10 प्रतिशत एसएल के प्रति सहनशील हैं. इस संदर्भ में IARI, दिल्ली के&nbsp;निदेशक डॉ.अशोक कुमार सिंह ने बताया कि, उत्तर-पश्चिमी भारत में चावल की खेती में प्रमुख चिंताओं में (A) घटता जल स्तर (B) चावल की रोपाई के लिए मजदूरों की कमी और (C) रोपाई के दौरान बाढ़ की स्थिति में ग्रीनहाउस गैस, मीथेन का उत्सर्जन शामिल है.</p>


                
    
            
                <div class="article-ad p-2 text-center mb-2">
            
            
             <!-- /21750466634/KJ_HINDI_INSIDE_ARTICLE_1 -->
            <div id="div-gpt-ad-1685456798873-0" style="min-width: 300px; min-height: 250px;">
              <script>
                googletag.cmd.push(function() { googletag.display('div-gpt-ad-1685456798873-0'); });
              </script>
            </div>
                
                
                
            </div>
            



                
    
<p>पूसा संस्थान द्ववारा विकसीत&nbsp;सीधी बुआई वाला चावल इन सभी चिंताओं का समाधान है. डीएसआर के तहत खरपतवार एक बड़ी समस्या है जिसे डीएसआर को सफल बनाने के लिए संबोधित करने की आवश्यकता है. इस दिशा में, ICAR-IARI, नई दिल्ली में ठोस अनुसंधान से रोबिनोविड बासमती चावल की दो किस्मों, पूसा बासमती 1979 और पूसा बासमती 1985 का विकास किया गया है,&nbsp;जो इमाजेथापायर 10% एसएल के प्रति सहनशील पहली गैर-जीएम शाकनाशी प्रतिरोधी बासमती चावल की किस्में हैं. इन्हें भारत में व्यावसायिक खेती के लिए जारी किया जाएगा.&nbsp;</p><div id="vuukle-quiz-container" style="display: flex; flex-wrap: wrap; background-color: transparent; justify-content: center; align-items: center; --darkreader-inline-bgcolor: transparent;" data-darkreader-inline-bgcolor=""><div id="vuukle-quiz" style="margin-top: 16px; margin-right: 1px;"><iframe height="0" name="quiz-iframe" id="quiz-iframe" title="Vuukle Quiz Widget" frameborder="0" scrolling="no" allow="web-share; clipboard-write" style="width: 100%; min-width: 100%; max-height: 265px; box-shadow: rgba(0, 0, 0, 0.3) 0px 0px 8px; border-radius: 6px; --darkreader-inline-boxshadow: rgba(0, 0, 0, 0.3) 0px 0px 8px; height: 275px;" data-darkreader-inline-boxshadow="" class="gm-added gm-lazy" src="https://cdn.vuukle.com/widgets/quiz.html?version=1.19.7"></iframe></div></div>


                
    
            
                <div class="ad text-center pb-2 mb-2">
                <!-- /21750466634/KJ_HIndi_300X250_Article_2 -->
            <div id="div-gpt-ad-1685592918480-0" style="min-width: 300px; min-height: 250px;">
              <script>
                googletag.cmd.push(function() { googletag.display('div-gpt-ad-1685592918480-0'); });
              </script>
            </div>
            </div>
            



                
    
<h2><strong>पूसा बासमती 1979</strong></h2>
<p>पुसा बासमती 1979 एक MAS से विकसित हर्बिसाइड सहिष्णु निकट-समानानुवांशिक पंक्ति है, जो बासमती चावल की किस्म "PB 1121" की है। इसमें Imazethapyr 10% एसएल सहिष्णुता प्रदान करने वाला परिवर्तित AHAS एलील होता है. इसके बीज से बीज तक परिपक्वता का समय 130 से 133 दिन का है. राष्ट्रीय बासमती परीक्षणों में 2 वर्षों के परीक्षण के दौरान सिंचित प्रतिरोपित अवस्था में इसकी औसत उपज 45.77 क्विंटल प्रति हेक्टेयर है.</p>


                
    
            
                <div class="article-ad p-2">
                <script async="" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
                <ins class="adsbygoogle" style="display:block; text-align:center;" data-ad-layout="in-article" data-ad-format="fluid" data-ad-client="ca-pub-3463764223457257" data-ad-slot="8544427592"></ins>
                <script>
                     (adsbygoogle = window.adsbygoogle || []).push({});
                </script>
            </div>
            



                
    
<h2>पूसा बासमती 1985</h2>
<div class="flex flex-grow flex-col max-w-full">
<div class="min-h-[20px] text-message flex flex-col items-start whitespace-pre-wrap break-words [.text-message+&amp;]:mt-5 juice:w-full juice:items-end overflow-x-auto gap-2" data-message-author-role="assistant" data-message-id="6fa47626-9e3f-497b-81dc-08aeeb3de6dd">
<div class="flex w-full flex-col gap-1 juice:empty:hidden juice:first:pt-[3px]">
<div class="markdown prose w-full break-words dark:prose-invert light">
<p>पुसा बासमती 1985 भी एक MAS से विकसित हर्बिसाइड सहिष्णु निकट-समानानुवांशिक पंक्ति है, जो बासमती चावल की किस्म "पीबी 1509" की है. धान की इस किस्म में&nbsp;Imazethapyr सहिष्णुता प्रदान करने वाला परिवर्तित AHAS एलील मौजूद है.&nbsp;इसके बीज से बीज तक की परिपक्वता का समय 115 से 120 दिन है। राष्ट्रीय बासमती परीक्षणों में 2 वर्षों के परीक्षण के दौरान सिंचित प्रतिरोपित अवस्था में इसकी औसत उपज 5.2 टन प्रति हेक्टेयर है।</p>
</div>
</div>
</div>
</div>


                
    
            
                <div class="article-ad p-2">
                <script async="" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
                <ins class="adsbygoogle" style="display:block; text-align:center;" data-ad-layout="in-article" data-ad-format="fluid" data-ad-client="ca-pub-3463764223457257" data-ad-slot="4605182586"></ins>
                <script>
                     (adsbygoogle = window.adsbygoogle || []).push({});
                </script>
            </div>
            



                
    
<p>उन्होंने इन दो फसल की किस्मों में खरपतवारों के प्रभावी प्रबंधन के लिए अपनाई जाने वाली आवश्यक सावधानियों के साथ डीएसआर के तहत इन दो चावल किस्मों की प्रथाओं के पैकेज के बारे में विस्तार से बताया है. ये दो किस्में व्यापक-स्पेक्ट्रम शाकनाशी के प्रति सहनशील होने के कारण, इमाज़ेथापायर 10% एसएल डीएसआर के तहत खरपतवारों के प्रभावी नियंत्रण में सहायक होंगी, जिससे बासमती चावल की खेती की लागत कम हो जाएगी. इससे चावल की खेती में श्रम की कमी, पानी और मीथेन उत्सर्जन की चुनौतियों से निपटने में भी मदद मिलेगी.</p>


                
    
            
                <div class="mt-3 mb-3">
                    <div class="mt-3 mb-3">
                    
                    
                    
                    
                  
                    <script async="" id="AV60f80399c91be72249341c0a" type="text/javascript" src="https://tg1.playstream.media/api/adserver/spt?AV_TAGID=60f80399c91be72249341c0a&amp;AV_PUBLISHERID=60a7b4f68e1cbe3066405d86"></script>
                    

                        
                    </div>
                </div>
            



                
    
<p>इस अवसर पर भारत सरकार के कृषि एवं किसान कल्याण मंत्रालय के कृषि आयुक्त डॉ. पीके सिंह ने भी चावल की खेती को टिकाऊ बनाने में इमेजेथापायर 10% एसएल के प्रति सहनशीलता जैसी तकनीकों के साथ इन उन्नत चावल किस्मों के महत्व पर जोर दिया. आईसीएआर, नई दिल्ली के एडीजी (बीज) डॉ. डीके यादव ने भी इस बात पर प्रकाश डाला कि बासमती चावल की ये दो किस्में देश के बासमती जीआई क्षेत्र के किसानों के लिए वरदान साबित होंगी. इस अवसर पर डॉ. सी. विश्वनाथन (संयुक्त निदेशक, अनुसंधान), डॉ. आरएन पडारिया (संयुक्त निदेशक, विस्तार), डॉ. गोपाल कृष्णन (आनुवांशिकी विभाग प्रमुख), डॉ. ज्ञानेंद्र सिंह (बीज उत्पादन इकाई प्रभारी), पूसा संस्थान के विभाग प्रमुख और वैज्ञानिक, किसान, बीज कंपनियां और मीडिया उपस्थित थे.</p>





                            
                            <div class="d-flex flex-column">
                            <span><strong>English Summary: </strong> pusa discovered two new varieties of Basmati paddy get bumper yield</span>
                            <span><strong>Published on: </strong>23 May 2024, 10:40 AM IST</span>
                        </div>
                        
                        </article>
"""

In [6]:
html = """<div class="news-list-wide shadow-sm">
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/" title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">
                                        <h2 title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत</h2>
                                        <p>किसानों को कम लागत और कम समय में&nbsp;धान की अच्छी पैदावार देने के उद्देश्य से&nbsp;पूसा संस्थान ने&nbsp;धान की 2 नई किस्में जारी की हैं. संस्थान ने&nbsp;कल…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/pusa-discovered-two-new-varieties-of-basmati-paddy-get-bumper-yield/" title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत">
                                        <img data-src="https://kjhindi.gumlet.io//media/87723/pusa-institute-new-delhi.jpg?width=220" alt="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत" title="पूसा ने बासमती धान की दो नई किस्में 1985 और 1979 खोजी,  बंपर मिलेगी उपज, जानें अन्य खासियत" loading="lazy" class="img-fluid shadow-sm lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io//media/87723/pusa-institute-new-delhi.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/ragi-madua-with-scientific-techniques-farmers-earn-good-profits-by-cultivating-ragi-ki-kheti/" title="वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा">
                                        <h2 title="वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा">वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा</h2>
                                        <p>Ragi Ki Kheti: रागी की खेती किसानों की आय बढ़ाने का सबसे अच्छा विकल्प है. इसकी खेती मुख्य रूप से मोटे अनाज के तौर पर की जाती है. दरअसल, रागी…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/ragi-madua-with-scientific-techniques-farmers-earn-good-profits-by-cultivating-ragi-ki-kheti/" title="वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा">
                                        <img data-src="https://kjhindi.gumlet.io//media/87720/ragi-ki-kheti.jpg?width=220" alt="वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा" title="वैज्ञानिक तकनीक से रागी (मडुआ) की खेती कर कमाएं अच्छा मुनाफा" loading="lazy" class="img-fluid shadow-sm lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io//media/87720/ragi-ki-kheti.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/chilli-cultivate-these-5-improved-varieties-of-green-chilli-in-june/" title="जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन">
                                        <h2 title="जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन">जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन</h2>
                                        <p>Chilli Cultivation: भारत में हरी मिर्च की खेती उड़ीसा, आंध्र प्रदेश, तामिलनाडु, कर्नाटक, महाराष्ट्र और राजस्थान में की जाती है. किसान हरी मिर्च की उन्नत किस्मों की खेती करके अधिक…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/chilli-cultivate-these-5-improved-varieties-of-green-chilli-in-june/" title="जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन">
                                        <img data-src="https://kjhindi.gumlet.io//media/87717/green-chilli-1.jpg?width=220" alt="जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन" title="जून में करें हरी मिर्च की इन 5 उन्नत किस्मों की खेती, प्रति हेक्टेयर मिलेगा 350 क्विंटल उत्पादन" loading="lazy" class="img-fluid shadow-sm lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io//media/87717/green-chilli-1.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/context-of-climate-change-farmers-can-do-direct-sowing-of-paddy-with-advanced-technology/" title="जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई ">
                                        <h2 title="जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई ">जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई </h2>
                                        <p>Paddy Cultivation: जलवायु परिवर्तन परिपेक्ष में आधुनिक कृषि प्रणालियों को अपनाना किसानों के लिए बेहद जरूरी है. धान आधारित फसल प्रणालियों के तहत 40 से 50% उत्पादन भारत के सिंधु-गंगा…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/context-of-climate-change-farmers-can-do-direct-sowing-of-paddy-with-advanced-technology/" title="जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई ">
                                        <img data-src="https://kjhindi.gumlet.io//media/87705/paddy-with-advanced-technology.jpg?width=220" alt="जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई " title="जलवायु परिवर्तन परिपेक्ष में उन्नत तकनीक के साथ किसान कर सकते हैं धान की सीधी बुवाई " loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb lazyloaded" src="https://kjhindi.gumlet.io//media/87705/paddy-with-advanced-technology.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/how-to-start-dragon-fruit-farming-in-india-dragon-fruit-different-varieties-cost-yield-and-profit/" title="Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान ">
                                        <h2 title="Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान ">Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान </h2>
                                        <p>Dragon Fruit Farming: ड्रैगन फ्रूट्स की खेती एक बार करने के बाद कई सालों तक फल मिलता रहता है. ड्रैगन फ्रूट्स की आधुनिक तरीके से खेती कर किसान सालाना लाखों…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/how-to-start-dragon-fruit-farming-in-india-dragon-fruit-different-varieties-cost-yield-and-profit/" title="Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान ">
                                        <img data-src="https://kjhindi.gumlet.io//media/87683/dragon-fruit.jpg?width=220" alt="Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान " title="Dragon Fruit Farming: ड्रैगन फ्रूट की व्यावसायिक खेती कर किसान कमाएं भारी मुनाफा, बस इन बातों का रखें ध्यान " loading="lazy" class="img-fluid shadow-sm lazyloaded gm-loaded gm-observing gm-observing-cb" src="https://kjhindi.gumlet.io//media/87683/dragon-fruit.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/maize-farming-in-kharif-season-is-more-profitable-than-paddy-farming/" title="खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा">
                                        <h2 title="खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा">खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा</h2>
                                        <p>मक्का की खेती देश में पूरे साल की जाती है. पानी की उपलब्धता होने पर अलग-अलग क्षेत्रों में किसान पूरे साल इसकी खेती करते हैं. अगर किसान धान के बजाए…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/maize-farming-in-kharif-season-is-more-profitable-than-paddy-farming/" title="खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा">
                                        <img data-src="https://kjhindi.gumlet.io//media/87682/makka.jpg?width=220" alt="खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा" title="खरीफ में मक्का लगाना है ज्यादा फायदेमंद, धान के मुकाबले मिलेगा दोगुना मुनाफा" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87682/makka.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/peanut-farming-tips-farmers-will-earn-more-profit-mungfali-ki-kheti/" title="मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!">
                                        <h2 title="मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!">मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!</h2>
                                        <p>Peanut Cultivation: यदि आप भी कम समय में खेती करके ज्यादा कमाई करना चाहते हैं, तो आपके लिए मूंगफली की खेती करना काफी शानदार विकल्प हो सकता है. इसकी खेती…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/peanut-farming-tips-farmers-will-earn-more-profit-mungfali-ki-kheti/" title="मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!">
                                        <img data-src="https://kjhindi.gumlet.io//media/87669/peanut-cultivation.jpg?width=220" alt="मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!" title="मूंगफली की खेती से किसानों की होगी मोटी कमाई, बस रखें इन बातों का ध्यान!" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87669/peanut-cultivation.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/peak-borer-infection-in-sugarcane-crop-damage-and-prevention/" title="गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान">
                                        <h2 title="गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान">गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान</h2>
                                        <p>Sugarcane Crop: गन्ने की खेती उत्तर प्रदेश, महाराष्ट्र, बिहार, पंजाब, हरियाणा, तमिलनाडु, गुजरात, आंध्रप्रदेश और उत्तराखंड में की जाती है. गन्ना किसानों के लिए इसकी खेती सुरक्षित तरीके से करनी…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/peak-borer-infection-in-sugarcane-crop-damage-and-prevention/" title="गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान">
                                        <img data-src="https://kjhindi.gumlet.io//media/87660/sugarcane-crop-tips.jpg?width=220" alt="गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान" title="गन्ने की फसल का इस खतरनाक कीट से ऐसे करें बचाव, वरना उठाना पड़ेगा भारी नुकसान" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87660/sugarcane-crop-tips.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/black-maize-cultivation-farmers-earn-huge-income-one-corn-cost-200-rupees/" title="काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये">
                                        <h2 title="काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये">काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये</h2>
                                        <p>Black Corn Farming: जायद सीजन की फसलों का बुवाई का समय चल रहा है और ऐसे में किसान अच्छी कमाई के लिए काली मक्का की खेती कर सकते हैं. इस…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/black-maize-cultivation-farmers-earn-huge-income-one-corn-cost-200-rupees/" title="काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये">
                                        <img data-src="https://kjhindi.gumlet.io//media/87646/black-corn-farming-1.jpg?width=220" alt="काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये" title="काला मक्का की खेती से किसानों की होगी ताबड़तोड़ कमाई, एक भुट्टे की कीमत है 200 रुपये" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87646/black-corn-farming-1.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/top-4-varieties-of-soybean-in-india-soyabean-ki-kheti/" title="सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज">
                                        <h2 title="सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज">सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज</h2>
                                        <p>Varieties of Soybean in India: सोयाबीन की खेती से अधिक पैदावार पाने के लिए किसानों को इसकी उन्नत किस्मों का चयन करना चाहिए. इसी क्रम में आज हम देश के…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/top-4-varieties-of-soybean-in-india-soyabean-ki-kheti/" title="सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज">
                                        <img data-src="https://kjhindi.gumlet.io//media/87638/portal-thumbnail-may-2024-2.jpg?width=220" alt="सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज" title="सोयाबीन की ये उन्नत किस्में किसानों की बढ़ाएंगी आय, मिलेगी 40 क्विटंल/हेक्टेयर उपज" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87638/portal-thumbnail-may-2024-2.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/how-to-buy-nsc-basmati-rice-seeds-at-cheap-rates-by-online-order-on-ondc-my-store/" title="NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज">
                                        <h2 title="NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज">NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज</h2>
                                        <p>NSC Basmati Rice Seeds: राष्ट्रीय बीज निगम लिमिटेड (एनएससी), भारतीय कृषि अनुसंधान संस्थान, पूसा द्वारा विकसित की गई बासमती धान की दो उन्नतशील प्रजातियां- पीबी-1692 एवं पीबी- 1509 को आकर्षक…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/how-to-buy-nsc-basmati-rice-seeds-at-cheap-rates-by-online-order-on-ondc-my-store/" title="NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज">
                                        <img data-src="https://kjhindi.gumlet.io//media/87636/nsc-basmati-rice-seeds.jpg?width=220" alt="NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज" title="NSC की बड़ी पहल, किसान अब घर बैठे ऑनलाइन आर्डर कर किफायती कीमत पर खरीद सकते हैं बासमती धान के बीज" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87636/nsc-basmati-rice-seeds.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/method-for-drumstick-farming-multi-year-vegetable-drumstick-cultivation/" title="सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा">
                                        <h2 title="सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा">सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा</h2>
                                        <p>सहजन बिहार में साल भर उगाये जाने वाला बहुवर्षीय सब्जी है. सहजन के लिए किसी विशेष देखभाल की आवश्यकता नहीं होती हैं. दक्षिण भारत में सहजन के फूल-फल-पत्ती का उपयोग…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/method-for-drumstick-farming-multi-year-vegetable-drumstick-cultivation/" title="सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा">
                                        <img data-src="https://kjhindi.gumlet.io//media/87607/portal-thumbnail-may-2024-2.jpg?width=220" alt="सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा" title="सहजन की खेती के लिए अपनाएं ये विधि, कम लागत में मिलेगा ज्यादा मुनाफा" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87607/portal-thumbnail-may-2024-2.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/multiple-cropping-with-maize-crop-in-hindi/" title="मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी">
                                        <h2 title="मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी">मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी</h2>
                                        <p>मक्का मुख्य रूप से खरीफ की फसल है. मक्का की अल्पकालिक किस्म 70 से 80 दिन में पक कर तैयार हो जाती है. इस प्रकार बहुफसली कार्यक्रम में इनका महत्व…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/multiple-cropping-with-maize-crop-in-hindi/" title="मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी">
                                        <img data-src="https://kjhindi.gumlet.io//media/87608/multicropping-with-maize.jpg?width=220" alt="मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी" title="मक्का के साथ बहूफसली खेती किसानों के लिए है बेहद लाभकारी, जानें पूरी जानकारी" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87608/multicropping-with-maize.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/basmati-paddy-top-5-varieties-more-yield-in-less-time/" title="बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार">
                                        <h2 title="बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार">बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार</h2>
                                        <p>Basmati Rice Varieties: खरीफ सीजन नजदीक है और इसमें धान की खेती का जाती है. धान की कुछ किस्में ऐसी भी है, जिसकी खेती से कम समय में अधिक फसल…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/basmati-paddy-top-5-varieties-more-yield-in-less-time/" title="बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार">
                                        <img data-src="https://kjhindi.gumlet.io//media/87588/best-basmati-varieties.jpg?width=220" alt="बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार" title="बासमती धान की टॉप 5 किस्में, जो कम समय में देती हैं 60 क्विंटल तक पैदावार" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87588/best-basmati-varieties.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>
                            <div class="news-item">
                                <div class="news-info">
                                    <a href="/farm-activities/use-of-vermiwash-in-the-field-of-agriculture-vermiwash-preparation-process/" title="जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि">
                                        <h2 title="जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि">जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि</h2>
                                        <p>वर्मीवाश के उपयोग से न केवल फसल से अधिक उपज पाने के लिए किया जाता है, बल्कि इसे किसान प्राकृतिक जैव कीटनाशक के रूप में भी प्रयोग किया जा सकता…</p>
                                    </a>
                                </div>
                                <div class="news-img">
                                    <a href="/farm-activities/use-of-vermiwash-in-the-field-of-agriculture-vermiwash-preparation-process/" title="जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि">
                                        <img data-src="https://kjhindi.gumlet.io//media/87584/portal-thumbnail-may-2024-4.jpg?width=220" alt="जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि" title="जैविक खेती में बेहद उपयोगी है वर्मीवाश, जानें प्रयोग, लाभ और बनाने की विधि" loading="lazy" class="img-fluid shadow-sm gm-loaded gm-observing gm-observing-cb ls-is-cached lazyloaded" src="https://kjhindi.gumlet.io//media/87584/portal-thumbnail-may-2024-4.jpg?w=220&amp;dpr=1.3">
                                    </a>
                                </div>
                            </div>

                </div>"""
